# 03 · Export FHIR Observation JSON (one per ADA test) + link PK via derivedFrom

In [9]:
from pathlib import Path
import pandas as pd, yaml, json

CFG = yaml.safe_load(open(str(Path.cwd().parents[0] / "config.yml")))
df = pd.read_csv(Path.cwd().parents[0] / "cdisc" / "adam_like" / "adimmun.csv")

loinc = CFG["fhir"]["loinc_ada_qual"]
pos = CFG["fhir"]["interpretation_codes"]["positive"]
neg = CFG["fhir"]["interpretation_codes"]["negative"]

outdir = Path.cwd().parents[0] / "interoperability" / "fhir_examples"
outdir.mkdir(parents=True, exist_ok=True)

def obs_id(r):
    return f"ADA_{r.USUBJID}_W{int(r.VISIT)}_{r.METHOD}"

def pk_obs_id(r):
    return f"PK_{r.USUBJID}_W{int(r.VISIT)}"

def to_obs(r):
    value_bool = True if r.ADASTAT == "POS" else False
    interp = pos if value_bool else neg
    return {
      "resourceType": "Observation",
      "id": obs_id(r),
      "status": "final",
      "code": {"coding": [{
        "system": "http://loinc.org", "code": loinc, "display": "Anti-drug antibody qualitative"
      }]},
      "subject": {"reference": f"Patient/{r.USUBJID}"},
      "effectiveDateTime": "2025-01-01",
      "valueBoolean": value_bool,
      "method": {"text": f"{r.METHOD} ADA Assay"},
      "interpretation": [{
        "coding": [{
          "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation",
          "code": interp
        }]
      }],
      "derivedFrom": [{"reference": f"Observation/{pk_obs_id(r)}"}],
      "extension": [{
        "url": "http://example.org/fhir/StructureDefinition/drug-tolerance-flag",
        "valueString": "PASS"
      }]
    }

count = 0
for _, r in df.iterrows():
    o = to_obs(r)
    with open(outdir / f"{o['id']}.json", "w") as f:
        json.dump(o, f, indent=2)
    count += 1
print("Wrote FHIR Observations:", count, "→", outdir)

Wrote FHIR Observations: 12720 → /Users/cmontefusco/Coding projects/dmpk-standards-interoperability/interoperability/fhir_examples
